Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [3]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

,player,starter,percentage
0,Montipo',1,90.0
1,Magnani,1,90.0
2,Coppola D.,1,90.0
3,Amione,1,80.0
4,Faraoni,1,65.0
...,...,...,...
465,Ranocchia F.,0,50.0
466,Gyasi,0,55.0
467,Shpendi S.,0,35.0
468,Destro,0,30.0


In [4]:
probables = probables.set_index('player')

In [5]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

,player1,player2,percentage
0,Faraoni,Terracciano F.,65.0
1,Djuric,Bonazzoli,55.0
2,Suslov,Hongla,60.0
3,Olivera,Mario Rui,65.0
4,Vlasic,Karamoh,60.0
5,Toljan,Pedersen,55.0
6,Ferrari G.,Tressoldi,60.0
7,Bajrami,Thorstvedt,60.0
8,Castellanos,Immobile,55.0
9,Rovella,Cataldi,60.0


In [6]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

,starter,percentage
player,,
Montipo',1,90.0
Magnani,1,90.0
Coppola D.,1,90.0
Amione,1,80.0
Faraoni,0.65,65.0
...,...,...
Ranocchia F.,0,50.0
Gyasi,0,55.0
Shpendi S.,0,35.0


In [7]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   